# Joining Data with pandas
1. Data merging basics
   - Inner join
   - One-to-many relationships
   - Merging multiple DataFrames
2. Merging Tables With Different Join Types
   - Left join
   - Outer join
   - Merging a table to itself
   - Merging on indexes
3. Advanced Merging and Concatenating
   - Filtering joins
   - Concatenate DataFrames together vertically
   - Verifying integrity
4. Merging Ordered and Time-Series Data
   - Using merge_ordered()
   - Using merge_asof()
   - Selecting data with .query()
   - Reshaping data with .melt()

NOTE: There are a large number of datasets in the `datasets/` folder. Many of these are Pickle files, which you can read using `pd.read_pickle(path_to_file)`. An example is included in the cell below.

In [1]:
# Import pandas
import pandas as pd

# Import some of the course datasets 
actors_movies = pd.read_csv("datasets/actors_movies.csv")
business_owners = pd.read_pickle("datasets/business_owners.p")
casts = pd.read_pickle("datasets/casts.p")

# Preview one of the DataFrames
casts

,movie_id,cast_id,character,gender,id,name
7,5,22,Jezebel,1,3122,Sammi Davis
8,5,23,Diana,1,3123,Amanda de Cadenet
9,5,24,Athena,1,3124,Valeria Golino
3,5,25,Elspeth,1,3125,Madonna
12,5,26,Eva,1,3126,Ione Skye
...,...,...,...,...,...,...
0,433715,3,Amber,0,1500111,Nicole Smolen
1,433715,4,BB,0,1734573,Kim Baldwin
2,433715,5,Sugar,0,1734574,Ariana Stephens
3,433715,6,Drew,0,1734575,Bryson Funk


# 1. Data merging basics

## Inner join
- Two dataframes must have a common columns then: 
  - df1_df2 = df1.merge(df2, on='common_column') 
  - the columns of df1 comes first. 
- If dataframes have the same column name then suffixes are used.
  - to control suffixes: df1.merge(df2, on='common_column', suffixes=('_suf1', '_suf2')) 
- Inner join has one to one relationship.

## One-to-many relationships
-  In a one-to-many relationship, every row in the left table is related to one or more rows in the right table.
-  The same syntaxes are used to deal with one-to-many relatonships.
- agg({'account':'count'}) is a Pandas DataFrame aggregation operation.
  -  account is the column name to aggregate.
  -  count is the aggregation function. 
  -  agg({'account':'count'}) says: “Count the number of unique account values in each group”.

## Merging multiple DataFrames
- single merge: df1.merge(df2, on=[cname1, cname2]), merging on two columns
- mergin multiple tables: df1.merge(df2, on=[cname1, cname2]).merge(df3, on=cname, suffixes=('_suf1', '_suf2'))

In [ ]:
### Inner join ###

#----------------------------------------------
# Yout first inner join

# Figure out what the most popular types of fuel used in Chicago taxis are. 
# Merge the taxi_owners and taxi_veh tables together on the vid column. 

# Merge the taxi_owners and taxi_veh tables setting a suffix
taxi_own_veh = taxi_owners.merge(taxi_veh, on='vid', suffixes=('_own','_veh'))

# Print the value_counts to find the most popular fuel_type
print(taxi_own_veh['fuel_type'].value_counts())

#-----------------------------------------------
# Inner joins and number of rows returned

# Merge the wards and census tables on the ward column
wards_census = wards.merge(census, on='ward')

# Print the shape of wards_census
print('wards_census table shape:', wards_census.shape)

# Print the first few rows of the wards_altered table to view the change 
print(wards_altered[['ward']].head())

# Merge the wards_altered and census tables on the ward column
wards_altered_census = wards_altered.merge(census, on='ward')

# Print the shape of wards_altered_census
print('wards_altered_census table shape:', wards_altered_census.shape)

# Print the first few rows of the census_altered table to view the change 
print(census_altered[['ward']].head())

# Merge the wards and census_altered tables on the ward column
wards_census_altered = wards.merge(census_altered, on='ward')

# Print the shape of wards_census_altered
print('wards_census_altered table shape:', wards_census_altered.shape)



In [2]:
### One-to-many relationships ###

#--------------------------------------

# Merge the licenses and biz_owners table on account
licenses_owners = licenses.merge(biz_owners, on='account')

# Group the results by title then count the number of accounts
counted_df = licenses_owners.groupby('title').agg({'account':'count'})

# Sort the counted_df in descending order
sorted_df = counted_df.sort_values('account', ascending=False)

# Use .head() method to print the first few rows of sorted_df
print(sorted_df.head())


In [ ]:
### Merging multiple DataFrames ###

#---------------------------------------------
# Total riders in a month

# TASK: Find the total number of rides provided to passengers passing through 
# the Wilson station (station_name == 'Wilson') when riding Chicago's public 
# transportation system on weekdays (day_type == 'Weekday') in July (month == 7).

# i) Merge the ridership and cal tables together, starting with the ridership table on the left.
# ii) Extend the previous merge to three tables by also merging the stations table.
# iii) Select the appropriate rows from the merged table and sum the rides column.

# Merge the ridership, cal, and stations tables
ridership_cal_stations = ridership.merge(cal, on=['year','month','day']) \
							.merge(stations, on='station_id')

# Create a filter to filter ridership_cal_stations
filter_criteria = ((ridership_cal_stations['month'] == 7) 
                   & (ridership_cal_stations['day_type'] == 'Weekday') 
                   & (ridership_cal_stations['station_name'] == 'Wilson'))

# Use .loc and the filter to select for rides
print(ridership_cal_stations.loc[filter_criteria])
print(ridership_cal_stations.loc[filter_criteria, 'rides'].sum())

#--------------------------------------------
# Three table merge

# i) Starting with the licenses table, merge to it the zip_demo table on the zip column. 
#    Then merge the resulting table to the wards table on the ward column.
# ii) Group the results of the merged table by the column alderman and find the median income.

# Merge licenses and zip_demo, on zip; and merge the wards on ward
print(licenses.head())
print(zip_demo.head())
print(wards.head())

licenses_zip_ward = licenses.merge(zip_demo, on='zip')\
            			.merge(wards, on='ward')
print(licenses_zip_ward.head())

# Print the results by alderman and show median income
print('')
print(licenses_zip_ward.groupby('alderman').head())
print(licenses_zip_ward.groupby('alderman').agg({'income':'median'}))

# Comment: You see that only a few aldermen represent businesses in areas
# where the median income is greater than $62,000, which is the median income for the state of Illinois.

#---------------------------------------------------
# One-to-many merge with multiple tables

# TASK: In the city of Chicago, start a business that uses goats to mow the lawn for other businesses. 
# For the location of the goat farm, it is required to a great deal of space and relatively 
# few businesses and people around to avoid complaints about the smell. Use three tables.

print(land_use.head())
print(census.head())
print(licenses.head())

# i) Merge land_use and census on the ward column and merge result with licenses including suffixes
land_cen_lic = land_use.merge(census, on='ward') \
                    .merge(licenses, on='ward', suffixes=('_cen','_lic'))
#print(land_cen_lic.head())

# ii) Group by ward, pop_2010, and vacant, then count the # of accounts
pop_vac_lic = land_cen_lic.groupby(['ward','pop_2010','vacant'], 
                                   as_index=False).agg({'account':'count'})

# iii) Sort pop_vac_lic by vacant, account, andpop_2010 in descending, ascending, and ascending order respectively.
sorted_pop_vac_lic = pop_vac_lic.sort_values(['vacant','account','pop_2010'], 
                                             ascending=[False,True,True])
print(sorted_pop_vac_lic.head())

# 2. Merging Tables With Different Join Types

## Left join
-  A left join returns all rows of data from the left table and only those rows from the right table where key columns match.
-  df1_df2 = df1.merge(df2, on='common_column', how='left') 
-  If your goal is to enhance or enrich a dataset, then you do not want to lose any of your original data. A left join will do that by returning all of the rows of your left table, while using an inner join may result in lost data if it does not exist in both tables.
-  A left join will return all of the rows from the left table. If those rows in the left table match multiple rows in the right table, then all of those rows will be returned. Therefore, the returned rows must be equal to if not greater than the left table. Knowing what to expect is useful in troubleshooting any suspicious merges.

## Other joins
- Right join:
    - df1.merge(df2, how='right', left_on='common_column_left', right_on='common_column_right') 
    - Both common columns have the same information with different column name (key column).
- Outer join:
    - An outer join will return all of the rows from both tables regardless if there is a match between the tables.
    - f1.merge(df2, on='common_columns', how='outer')
    - 
## Merging a table to itself
- One can use all merging methods above.
- When to merge a table to itself:
    - hierarchical relationship
    - sequential relationships
    - graph data

## Merging on indexes
- we learn how to merge tables using their indexes
- setting an index: df = pd.read_csv('df_path', index_col=['col_name'])
- merging on index:
    - identical as before, however we merge on the index level name.
    - df1_df2 = df1.merge(df2, on='index_level_name', how='left') 
- multiIndex merger:
    - df = pd.read_csv('df_path', index_col=['col_name1', 'col_name2'])
    - df1.merge(df2, on=['col_name1', 'col_name2']) : both indexes must match in inner join.
- index merge with left_on and right_on:
    - df1.merge(df2, left_on='id', left_index=Trues, right_on='movie_id', right_index=True)
    - if dfs are merged using index level names in left_on, one must set left_index to True. 

In [ ]:
### Left join ####

#-------------------------------------------
# Counting missing rows with left join

# TASK: Find out which movie still has the missing financial data.

# Merge the movies table with the financials table with a left join
movies_financials = movies.merge(financials, on='id', how='left')
print(movies_financials.head())
print(movies_financials.shape)

# Count the number of rows in the budget column that are missing
number_of_missing_fin = movies_financials['budget'].isna().sum()

# Print the number of movies missing financials
print(number_of_missing_fin)

#---------------------------------------------
# Enriching a dataset

# See difference between inner and left join

# Merge the toy_story and taglines tables with a left join
toystory_tag = toy_story.merge(taglines, on='id', how='left')

# Print the rows and shape of toystory_tag
print(toystory_tag)
print(toystory_tag.shape)

# Merge the toy_story and taglines tables with a inner join
toystory_tag = toy_story.merge(taglines, on='id')

# Print the rows and shape of toystory_tag
print(toystory_tag)
print(toystory_tag.shape)


In [ ]:
### Other joins ###

#---------------------------------------------
# Right join to find unique movies

# Most of the recent big-budget science fiction movies can also be classified as action movies. 
# Given a table of science fiction movies (scifi_movies) and of action movies (action_movies). 
# Find which movies are considered only science fiction movies. 

# Merge action_movies to the scifi_movies with right join
action_scifi = action_movies.merge(scifi_movies, on='movie_id', how='right',
                                   suffixes=('_act','_sci'))
print(action_scifi.head())

# From action_scifi, select only the rows where the genre_act column is null
scifi_only = action_scifi[action_scifi['genre_act'].isnull()]
print(scifi_only.head())

# Merge the movies and scifi_only tables with an inner join
print(movies.head())
movies_and_scifi_only = movies.merge(scifi_only, left_on='id', right_on='movie_id')

# Print the first few rows and shape of movies_and_scifi_only
print(movies_and_scifi_only.head())
print(movies_and_scifi_only.shape)

#---------------------------------------------
# Popular genres with right join

# What are the genres of the most popular movies? 

# Use right join to merge the movie_to_genres and pop_movies tables
# The right join ensures that all of the pop_movies are analysed.
print(movie_to_genres.head())
print(pop_movies.head())
genres_movies = movie_to_genres.merge(pop_movies, how='right', 
                                      left_on='movie_id', 
                                      right_on='id')

print(genres_movies.head())
print(pop_movies.shape, genres_movies.shape)

# Count the number of genres
genre_count = genres_movies.groupby('genre').agg({'id':'count'})

# Plot a bar chart of the genre_count
genre_count.plot(kind='bar')
plt.show()

#---------------------------------------------
# Using outer join to select actors
# In outer join, there is no loose of data.

# Find actors who did not act in both 'Iron man' movies.

# Merge iron_1_actors to iron_2_actors on id with outer join using suffixes
iron_1_and_2 = iron_1_actors.merge(iron_2_actors,
                                     on ='id', how='outer',
                                     suffixes=('_1','_2'))
print(iron_1_and_2)

# Create an index that returns true if name_1 or name_2 are null
m = ((iron_1_and_2['name_1'].isnull()) | 
     (iron_1_and_2['name_2'].isnull()))

# Print the first few rows of iron_1_and_2
print(iron_1_and_2[m].head())


In [ ]:
### Merging a table to itself ###
#---------------------------------------------
# Self join

# Merge the crews table to itself
print(crews.head())
crews_self_merged = crews.merge(crews, on='id', suffixes=('_dir', '_crew'))
print(crews_self_merged.iloc[:10])

# Create a Boolean index to select rows from the left table with the job of 'Director' 
# and avoids rows with the job of 'Director' in the right table.
boolean_filter = ((crews_self_merged['job_dir'] == 'Director') & 
     (crews_self_merged['job_crew'] != 'Director'))
direct_crews = crews_self_merged[boolean_filter]
print(direct_crews.head())

# With the output, you can quickly see different movie directors 
# and the people they worked with in the same movie.

In [ ]:
### Merging on Indexes ###

#---------------------------------------------
# Index merge for movie ratings 

# Merge to the movies table the ratings table on the index
# Keep all rows from the movies table.
print(movies.head())
print(ratings.head())

movies_ratings = movies.merge(ratings, on='id', how='left')

# Print the first few rows of movies_ratings
print(movies_ratings.head())

#---------------------------------------------
# Do sequels earn more?

# TASK: find out which movie sequels earned the most compared to the original movie.
# Use a modified version of the sequels and financials tables where their index is the movie ID.

# Merge sequels and financials on index id, including all the rows from the sequels
print(sequels.head())
print(financials.head())

sequels_fin = sequels.merge(financials, on='id', how='left')
print(sequels_fin.head())

# Self merge with suffixes as inner join,
# where the left and right tables merge on sequel and id respectively 
# with suffixes equal to ('_org','_seq')
orig_seq = sequels_fin.merge(sequels_fin, how='inner', left_on='sequel', 
                             right_on='id', right_index=True,
                             suffixes=('_org','_seq'))
print(orig_seq)

# Add calculation to subtract revenue_org from revenue_seq 
orig_seq['diff'] = orig_seq['revenue_seq'] - orig_seq['revenue_org']

# Select the title_org, title_seq, and diff columns of orig_seq 
titles_diff = orig_seq[['title_org', 'title_seq', 'diff']]
print(titles_diff.head())

# Sort by titles_diff by diff in descending order 
print(titles_diff.sort_values('diff', ascending=False).head())


 # 3. Advanced Merging and Concatenating

## Filtering joins
- No direct support for filtering joins
- **Mutating joins** (that is what we have seen): combines data from two tables based on matching observations in both tables.
- **Filtering joins**: filtering observations from one table based on whether or not they match an observation in another table.
- **Semi join**: filters the left table down to those observations that have a match in the right table.
    - returns the intersections between the table, similar to inner join.
    - returns only columns from the left table and not the right one, unlike inner join.
    - no duplicate rows from the left table, even if there is one-to-many relationships.
- Steps of semi join:
    1. merge both tables on key column using inner join.
       df1_df2 = df1.merge(df2, on='common_col')
    2. search for rows from the left table belong in the final result creating a filter (using .isin() method).
       filter = df1[ df1['common_col'].isin(df1_df2['common_col']) ]
    3. subset the rows of the left table with that filter.
       df1_in_df2 = df1[filter]
- See the steps of semi join in exercises...
- **Anti join**:
    - return the left table excluding the intersection.
    - returns only columns from the left table and not the right one
- Steps of anti join:
    1. use a left join returning all of the rows from the left table: 
       df1_df2 = df1.merge(df2, on='common_col', how='left', indicator=True)
    2. select the rows that only appeared in the left table using "loc" accessor and "_merge" column.
       only_df1 = df1_df2.loc[df1_df2['_merge']=='left_only', 'common_col']
    3. use the isin() method to filter for the rows with the above filter. As last find the observations in df1 which has no matching with df2.
       df1_not_in_df2 = df1[df1['common_col'].isin()only_df1]
- .merge(...,...., indicator=True): 
    - returns a _merge columns
    - tells the source of each row

Comment: Looked a little complicated to me! Check the exercises carefully!

## Concatenate DataFrames together vertically
- So far, the horizontal merge of two tables are discussed. How about vertical merge?
- pandas .concat() method can concatenate both vertically or horizontally, but in this lesson, we'll focus on vertical concatenation.
- .concat(axis=0): to concatenate vertically
- Basic concatenation:
    - same column names
    - pd.concat([df1,df2,df3])
    - axis=0 is for vertical concatenation and it is default.
    - each tables's index value is retained. 
    - if no valueable information at indexing, then ignore :  pd.concat([df1,df2,df3], ignore_index=True)
    - setting labels to original tables: pd.concat([df1,df2,df3], ignore_index=False, keys=['_first', '_second','_third'])
    - these labels are not suffixes, they are labels to keys argument. This gives multi-index, with the label on the first level.
- Different column names:
    - pd.concat([df1,df2], sort=True): 
    - sort=True: sorting different column names alphabetically.
    - the values in different column name are written as NaN for the tables not including that column name.
    - Only mathcing columns:
        - pd.concat([df1,df2], join='inner')
        - join='outer' is default, that is why .concat() method includes all of the columns as default.
        - when join='inner', sort argument has no effect.

## Verifying integrity
- possible merging issue:
    - unintentional one-to-many relationships
    - unintentional many-to-many relationships
- possible vertical concatenating issue:
    - dublicate records possibly unintentionally introduced
- Validating merges: .merge(validate= ...), 'one_to_one','one_to_many','many_to_one','many_to_many' 
    - df1.merge(df2, on='common_col', validate='one_to_one')
    - if not one_to_one relation, then merge error raises.
- Verifying concatenations:
    - pd.concat([df1, df2], veirfy_integrity=False)
    - check whether the new concatenated index contains duplicates
    - check only index values not columns
    - the default value is False
    - when it is set True, it raises an error in the pesence of duplicate values
- Why verify integrity:
    - real world data is not clean
    - real world data may not have the expected structure
- what to do: 
    - fix incorret data
    - drop duplicate rows

In [ ]:
### Filterin gjoins ###

#--------------------------------------------------
# Performing an anti join

# TASK: filter the employee table by a table of top customers, returning only those employees who are not assigned to a customer.

# Step i) Merge employees and top_cust with a left join, setting indicator argument to True.
empl_cust = employees.merge(top_cust, on='srid', how='left', indicator=True)
print(employees.head())
print(top_cust.head())
print(empl_cust)

# Step ii) Select the srid column where _merge is left_only
srid_list = empl_cust.loc[empl_cust['_merge']=='left_only', 'srid']
print(srid_list)

# Step iii) Subset the left table using the filter
print(employees[employees['srid'].isin(srid_list)])

#--------------------------------------------------
# Performing a semi join

# TASK: find the top revenue-generating non-musical tracks.
print(non_mus_tcks.head(), non_mus_tcks.shape)
print(top_invoices.head(),top_invoices.shape)

# Step i)  Merge the non_mus_tcks and top_invoices tables on tid
tracks_invoices = non_mus_tcks.merge(top_invoices, on='tid', how='inner')
print(tracks_invoices.head(), tracks_invoices.shape)

# Step ii and iii) Use .isin() to subset non_mus_tcks to rows with tid in tracks_invoices
top_tracks = non_mus_tcks[non_mus_tcks['tid'].isin(tracks_invoices['tid'])]
print(top_tracks, top_tracks.shape)

# Group the top_tracks by gid and count the tid rows
cnt_by_gid = top_tracks.groupby(['gid'], as_index=False).agg({'tid':'count'})

# INFO: In pandas’ groupby method, as_index=False tells that 
# do not use grouping column as the index of the resulting DataFrame.

# Merge the genres table to cnt_by_gid on gid and print
print(cnt_by_gid.merge(genres, on='gid'))

# COMMENT : In this exercise, you replicated a semi join to filter the table of tracks by the table of invoice items to find the top revenue non-musical tracks. With some additional data manipulation, you discovered that _'TV-shows'_ is the non-musical genre that has the most top revenue-generating tracks.

In [ ]:
### Concatenate DataFrames together vertically ###

#--------------------------------------------------
# Concatenation basics

# A few tables of data with musical track info (tracks_master, tracks_ride, and tracks_st) 
# for different albums from the metal band, Metallica. Try various features of the .concat() method

print(tracks_master)
print(tracks_ride)
print(tracks_st)

# i) Concatenate the tracks,  in that order, setting sort to True.
tracks_from_albums = pd.concat([tracks_master, tracks_ride, tracks_st], sort=True)
print(tracks_from_albums)

# ii) Concatenate the tracks so the index goes from 0 to n-1
tracks_from_albums = pd.concat([tracks_master, tracks_ride, tracks_st],
                               ignore_index=True, sort=True)
print(tracks_from_albums)

# iii) Concatenate the tracks, show only columns names that are in all tables
tracks_from_albums = pd.concat([tracks_master, tracks_ride, tracks_st],
                               join='inner', sort=True)
print(tracks_from_albums)

#-----------------------------------------------------
# Concatenating with keys

# TASK: The music streaming company needs  assistance in analyzing sales. 
# Find out which month saw the highest average invoice total. 
# Given three tables with invoice data (inv_jul, inv_aug, and inv_sep).

print(inv_jul.head())
print(inv_aug.head())
print(inv_sep.head())

# Concatenate the tables and add keys
inv_jul_thr_sep = pd.concat([inv_jul, inv_aug, inv_sep], 
                            keys=['7Jul', '8Aug', '9Sep'])
#print(inv_jul_thr_sep)

# Group the invoices by the index keys and find avg of the total column
avg_inv_by_month = inv_jul_thr_sep.groupby(level=0).agg({'total':'mean'})

# Bar plot of avg_inv_by_month
avg_inv_by_month.plot(kind='bar')
plt.show()

In [ ]:
### Verifying integrity ###
#-----------------------------------------------------
# Concatenate and merge to find common songs

# Given files for a popular playlist in the classical music genre in 2018 and 2019
# and given a similar set of files for the most popular pop music genre playlist in 2018 and 2019. 
# TASK: i) concatenate the files to make a classical playlist table and overall popular music table. 
#       ii) filter the classical music table using a semi join to return only the most popular classical musics.

# i)
# Concatenate the classic tables vertically with index from 0 to n-1
classic_18_19 = pd.concat([classic_18, classic_19], ignore_index=True)

# Concatenate the pop tables vertically with index from 0 to n-1
pop_18_19 = pd.concat([pop_18, pop_19], ignore_index=True)

# ii) semi join:
# Merge classic_18_19 with pop_18_19
# default 'inner' keeps only common columns
classic_pop = classic_18_19.merge(pop_18_19, on='tid', how='inner')
print(classic_18_19)
print(pop_18_19)
print(classic_pop, classic_pop.shape)

# Using .isin(), filter classic_18_19 rows where tid is in classic_pop
popular_classic = classic_18_19[classic_18_19['tid'].isin(classic_pop['tid'])]

# Print popular chart
print(popular_classic, popular_classic.shape)


# 4. Merging Ordered and Time-Series Data

## Using merge_ordered()
- the result is similar to standard merge method with an outer join, but the result is sorted.
- .merge() method:
    - columns to join on: on, left_on, right_on
    - type of join: how(left, right, inner, outer), **default** inner
    - overlapping column names: suffixes
    - **calling the method**: df1.merge(df2)
- .merge_ordered():
    - columns to join on: on, left_on, right_on
    - type of join: how(left, right, inner, outer), **default** outer
    - overlapping column names: suffixes
    - **calling the method**: pd.merge_ordered(df1,df2)
- Forward fill:
    - fills missing data with previous value.
    - fill_method='ffill'
- When to use merge_ordered():
    - ordered data/ time series
    - filling in missing values (the fill forward feature)

## Using merge_asof()
- similar features as merge_ordered() with left join
- match on the nearest key column and not exact matches
- **important**: so that whatever columns you merge on must be sorted!
- pd.merge_asof(df1, df2, on=...., suffixes=....)
- selecting values: .....
    - ...., direction='forward'): forward direction
    - 'backward' is default, or 'nearest' is also an option.
- when to use merge_asof():
    - data sampled from a process and the dates or times may not exactly align. 
    - developing a time-series training set, where any events from the future is asked to be visible before that time.
    
## Selecting data with .query()
- The query() method accepts an input string:
    - Input strings used to select rows to return from the table. 
    - Input string similar to statement after WHERE clause of a SQL statement. 
- Querying on conditions: df.query('condition'), df.query('condition1 and/or condition2')
- single quotes to start the conditional statement, double quotes around the word!!!
 
## Reshaping data with .melt()
- Unpivot the dataset (writes it in long format which is more computer-friendla)
- df.melt(id_vars=['preserved_col1', 'pre_col2'], value_vars=['unpivoted_col1','unp_col2']):
    - id_vars: identifier variables.
    - id_vars are column names in the original dataset that we do not want to change.
    - value_vars: controls unpivoted columns
    - the order of value_vars is kept in the result.
    - ... var_name='...', value_name=''...)
    -  var_names sets the name of variable column
    -  value_names sets the name of value column.

In [ ]:
###  Using merge_ordered() ###

#--------------------------------------------------
# Correlation between GDP and S&P500

# Analyze stock returns from the S&P 500. There may be a relationship 
# between the returns of the S&P 500 and the GDP of the US.
print(sp500.head())
print(gdp.head())

# Use merge_ordered() to merge gdp and sp500 using left join on year and date
gdp_sp500 = pd.merge_ordered(gdp, sp500, left_on='year', right_on='date', 
                             how='left')
print(gdp_sp500.head())


# Use merge_ordered() to merge gdp and sp500,  similar to before, and forward fill missing values
gdp_sp500 = pd.merge_ordered(gdp, sp500, left_on='year', right_on='date', 
                             how='left', fill_method='ffill')
print (gdp_sp500)

# Subset  the gdp_sp500 table selecting the gdp and returns columns
gdp_returns = gdp_sp500[['gdp','returns']]
print(gdp_returns)

# Print the correlation matrix of the gdp_returns sing the .corr() method.
print (gdp_returns.corr())

#------------------------------------------------------------
# Phillips curve using merge_ordered()

# Economic Theory: Developed by A. W. Phillips which states 
# that inflation and unemployment have an inverse relationship. 
# The theory claims that with economic growth comes inflation, 
# which in turn should lead to more jobs and less unemployment.

# Given two tables of data from the U.S. Bureau of Labor Statistics, 
# containing unemployment and inflation data over different periods.
# Tables with different frequencies:
# inflation: a data entry every six months,
# unemployment: a data entry every month. 

print(unemployment.head(), unemployment.shape)
print(inflation.head(), inflation.shape)

# Use merge_ordered() to merge inflation, unemployment on date with inner join
inflation_unemploy = pd.merge_ordered(inflation, unemployment, on='date', how='inner')

# Print inflation_unemploy 
print(inflation_unemploy)

# Plot a scatter plot of unemployment_rate on the horizontal axis and cpi (inflation) on the vertical axis.
inflation_unemploy.plot(x='unemployment_rate', y='cpi', kind='scatter')
plt.show()

#------------------------------------------------------------
# merge_ordered() caution, multiple columns

# NOTE: When using merge_ordered() to merge on multiple columns,
# the order is important when you combine it with the forward fill feature. 

# Task: Merge GDP and population data from the World Bank for Australia and Sweden.
print(pop.head(), pop.shape)  # yearly data
print(gdp.head(), gdp.shape)  # quarterly data

# Merge gdp and pop on date and country with fill feature and notice rows 2 and 3
ctry_date = pd.merge_ordered(gdp, pop, on=['date','country'], 
                             fill_method='ffill')
print(ctry_date)

# Merge gdp and pop on country and date (reverse of step 1)  with fill
date_ctry = pd.merge_ordered(gdp, pop, on=['country','date'], fill_method='ffill')
print(date_ctry)

# COMMENT: Notice that in this exercise, just because of the order of merging columns
# and then applying fill feature, Swedens's info is used to fill the missing values of Australia.

In [ ]:
### Using merge_asof() ####

#--------------------------------------------------------
# Using merge_asof() to study stocks

# Given a feed of stock market prices that are recorded roughly every 5 minutes. 
# Price logs for three banks, JP Morgan (JPM), Wells Fargo (WFC), and Bank Of America (BAC). 

# TASK: How does the price change of the two other banks compare to JP Morgan?

print(jpm.head(), jpm.shape)
print(wells.head(), wells.shape)
print(bac.head(), bac.shape)

# Use merge_asof() to merge jpm (left table) and wells on the date_time column
# with the nearest times are matched, and with suffixes=('', '_wells')
jpm_wells = pd.merge_asof(jpm, wells, on='date_time', direction='nearest',
                            suffixes=('', '_wells'))
print(jpm_wells)

# Use merge_asof() to merge jpm_wells (left table) and bac together on the date_time column
jpm_wells_bac = pd.merge_asof(jpm_wells, bac, on='date_time', direction='nearest',
                            suffixes=('_jpm', '_bac'))
print(jpm_wells_bac)

# Use pandas .diff() method to compute the price change over time.
price_diffs = jpm_wells_bac.diff()
print(price_diffs)

# Plot the price diff of the close of jpm, wells and bac only
price_diffs.plot(y=['close_jpm', 'close_wells', 'close_bac'])
plt.show()

# NOTE: .merge_asof() can be used for fuzzy matches as here.

#---------------------------------------------------
# Using merge_asof() to create dataset

# The merge_asof() function can be used to create datasets where you have a table of start and stop dates.

# Given gdp, a table of quarterly GDP values of the US during the 1980s, and
# the table recession, holding the starting date of every US recession since 1980, 
# and the date when the recession was declared to be over.

# TASK: Use merge_asof() to merge the tables and create a status flag if a quarter was during a recession. 

print(gdp.head(), gdp.shape)
print(recession.head(), recession.shape)

# Merge gdp and recession on date using merge_asof()
gdp_recession = pd.merge_asof(gdp, recession, on='date')
print(gdp_recession)

# Create a list based on the row value of gdp_recession['econ_status'].
# for each row if the gdp_recession['econ_status'] value is equal to 'recession' then enter 'r' else 'g'.
is_recession = ['r' if s=='recession' else 'g' for s in gdp_recession['econ_status']]

# Plot a bar chart of gdp_recession
gdp_recession.plot(kind='bar', y='gdp', x='date', color=is_recession, rot=90)
plt.show()

In [ ]:
### Selecting data with .query() ###

#--------------------------------------------------
# Subsetting rows with .query()

# Revisit GDP and population data for Australia and Sweden.

print(gdp.head(), gdp.shape)
print(pop.head(), pop.shape)

# Merge gdp and pop on date and country with fill
gdp_pop = pd.merge_ordered(gdp, pop, on=['country','date'], fill_method='ffill')
print(gdp_pop)

# Add a column named gdp_per_capita to gdp_pop that divides the gdp by pop
gdp_pop['gdp_per_capita'] = gdp_pop['gdp'] / gdp_pop['pop']
print(gdp_pop)

# Pivot data so gdp_per_capita, where index is date and columns is country
gdp_pivot = gdp_pop.pivot_table('gdp_per_capita', 'date', 'country')
print(gdp_pivot)

# Select dates equal to or greater than 1991-01-01
recent_gdp_pop = gdp_pivot.query('date>="1991-01-01"')
print(recent_gdp_pop)

# Plot recent_gdp_pop
recent_gdp_pop.plot(rot=90)
plt.show()


In [ ]:
### Reshaping data with .melt() ###

#--------------------------------------------------------
# Using .melt() to reshape government data

# Given a table of US unemployment rate data from the US Bureau of Labor Statistics
# in an easy-to-read format - it has a separate column for each month, and each year is a different row.

print(ur_wide)

# unpivot everything besides the year column
ur_tall = ur_wide.melt(id_vars='year', var_name = 'month', value_name='unempl_rate')
print(ur_tall)

# Create a date column using the month and year columns of ur_tall
ur_tall['date'] = pd.to_datetime(ur_tall['year'] + '-' + ur_tall['month'])
print(ur_tall)

# Sort ur_tall by date in ascending order
ur_sorted = ur_tall.sort_values('date', ascending=True)
print(ur_sorted)

# Plot the unempl_rate by date
ur_sorted.plot(x='date', y='unempl_rate')
plt.show()

# COMMENT: .....

#------------------------------------------------------
# Using .melt() for stocks vs bond performance

# TASK: Confirm that the price of bonds is inversely related to the price of stocks.

# Given a table of percent change of the US 10-year treasury bond price in a wide format.
# Merge this table with a table of the percent change of the Dow Jones Industrial stock index price. 

print(ten_yr)
print(dji)

# Use melt on ten_yr, unpivot everything besides the metric column
bond_perc = ten_yr.melt(id_vars='metric', var_name='date', value_name='close')
print(bond_perc)

# Use query on bond_perc to select only the rows where metric=close
bond_perc_close = bond_perc.query('metric=="close"')
print(bond_perc_close)

# Merge (ordered) dji and bond_perc_close on date with an inner join
dow_bond = pd.merge_ordered(dji, bond_perc_close, on='date', 
                            how='inner', suffixes=('_dow', '_bond'))
print(dow_bond)

# Plot only the close_dow and close_bond columns
dow_bond.plot(y=['close_dow', 'close_bond'], x='date', rot=90)
plt.show()

# COMMENT: The bond and stock prices are inversely correlated.